In [22]:
import tensorflow as tf
import numpy as np



In [23]:
EPOCHS = 100

#모델 정의

In [24]:
class ConvLNReluBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(ConvLNReluBlock, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size, padding = 'same', use_bias=False)#레이어 정규화라서 bias일단꺼졍
        self.ln = tf.keras.layers.LayerNormalization()
    
    def call(self,x,trianing=False,mask = None):
        #배치 정규화와 다르게 training은 false여도 된다.
        x = self.conv(x)
        x = self.ln(x)
        return tf.nn.relu(x)

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        # self.conv1_1 = tf.keras.layers.Conv2D(16,(3,3),padding='same',activation='relu')
        # self.conv1_2 = tf.keras.layers.Conv2D(16,(3,3),padding='same',activation='relu')
        # self.pool1 = tf.keras.layers.MaxPool2D((2,2))

        # self.conv2_1 = tf.keras.layers.Conv2D(32,(3,3),padding='same',activation='relu')
        # self.conv2_2 = tf.keras.layers.Conv2D(32,(3,3),padding='same',activation='relu')
        # self.pool2 = tf.keras.layers.MaxPool2D((2,2))

        # self.conv3_1 = tf.keras.layers.Conv2D(64,(3,3),padding='same',activation='relu')
        # self.conv3_2 = tf.keras.layers.Conv2D(64,(3,3),padding='same',activation='relu')

        # self.flatten = tf.keras.layers.Flatten()
        # self.dense1 = tf.keras.layers.Dense(1024,activation = 'relu')
        # self.dense2 = tf.keras.layers.Dense(10, activation = 'softmax')

        #layer normalization 버전
        self.conv1_1 = ConvLNReluBlock(16,(3,3))
        self.conv1_2 = ConvLNReluBlock(16,(3,3))
        self.pool1 = tf.keras.layers.MaxPool2D((2,2))

        self.conv2_1 = ConvLNReluBlock(32,(3,3))
        self.conv2_2 = ConvLNReluBlock(32,(3,3))
        self.pool2 = tf.keras.layers.MaxPool2D((2,2))

        self.conv3_1 = ConvLNReluBlock(64,(3,3))
        self.conv3_2 = ConvLNReluBlock(64,(3,3))

        self.flatten = tf.keras.layers.Flatten()
        #가중치 정규화
        self.dense1 = tf.keras.layers.Dense(1024,activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))
        self.dense2 = tf.keras.layers.Dense(10, activation = 'softmax', kernel_regularizer=tf.keras.regularizers.l2(0.01))


    def __call__(self, x, training = False, mask = None):
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)

        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)

        x = self.conv3_1(x)
        x = self.conv3_2(x)

        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)


#데이터셋


In [25]:
cifar = tf.keras.datasets.cifar10
(x_train, y_train),(x_test, y_test) = cifar.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32).prefetch(1024)#하드에서 미리가져와서 돌려줌
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32).prefetch(1024)

#Keras API 모델 학습

In [26]:
model = MyModel()
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data = test_ds,epochs = EPOCHS)


Epoch 1/100
1563/1563 [==============================] - 18s 12ms/step - loss: 2.1966 - accuracy: 0.3985 - val_loss: 1.4284 - val_accuracy: 0.5288
Epoch 2/100
1563/1563 [==============================] - 18s 12ms/step - loss: 1.3048 - accuracy: 0.5796 - val_loss: 1.2104 - val_accuracy: 0.6100
Epoch 3/100
1563/1563 [==============================] - 18s 11ms/step - loss: 1.1242 - accuracy: 0.6445 - val_loss: 1.1575 - val_accuracy: 0.6321
Epoch 4/100
1563/1563 [==============================] - 18s 11ms/step - loss: 1.0167 - accuracy: 0.6832 - val_loss: 0.9984 - val_accuracy: 0.6901
Epoch 5/100
 697/1563 [============>.................] - ETA: 9s - loss: 0.9562 - accuracy: 0.7056

KeyboardInterrupt: ignored